In [9]:
import pandas as pd
import numpy as np
import sklearn
from keras.models import Sequential
from keras.layers import Dense, Dropout, Masking, Embedding
from matplotlib import pyplot as plt
from ipywidgets import interact
import pickle
from IPython.display import clear_output
plt.rcParams['figure.dpi'] = 200

In [10]:
# file_headers = ["/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext1",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext2",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext3",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext4",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext1",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext2",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext3",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext4",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve1",
#                 "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve2"]

file_headers = ["/Volumes/Flash/combined_data/far_right_up_and_back",
                "/Volumes/Flash/combined_data/far_left_up_and_back",
                "/Volumes/Flash/combined_data/up_and_back3",
                "/Volumes/Flash/combined_data/sweep1",
                "/Volumes/Flash/combined_data/sweep2",
                "/Volumes/Flash/combined_data/random",
                "/Volumes/Flash/combined_data/module1_fullext1",
                "/Volumes/Flash/combined_data/module1_fullext2",
                "/Volumes/Flash/combined_data/module1_fullext3",
                "/Volumes/Flash/combined_data/module1_fullext4",
                "/Volumes/Flash/combined_data/module2_fullext1",
                "/Volumes/Flash/combined_data/module2_fullext2",
                "/Volumes/Flash/combined_data/module2_fullext3",
                "/Volumes/Flash/combined_data/module2_fullext4",
                "/Volumes/Flash/combined_data/s_curve1",
                "/Volumes/Flash/combined_data/s_curve1_part2",
                "/Volumes/Flash/combined_data/s_curve2",
                "/Volumes/Flash/combined_data/s_curve2_part2"]

CSV_SFX = ".csv"
MARKERS_SFX = "_markers"
POLY_SFX = "_poly"
M10Y = "M10Y"

In [11]:
# distance goes from 22-38. 16 cm of extension at least which is pretty good.

In [19]:
poly_df_list = []
marker_df_list = []
data_df_list = []
idx_df_list = []

running_idx = 0
end_idx = []

for header in file_headers:
    data_file = header + CSV_SFX
    marker_file = header + MARKERS_SFX + CSV_SFX
    #poly_file = header + POLY_SFX + CSV_SFX
    
    data_df = pd.read_csv(data_file)
    marker_df = pd.read_csv(marker_file)
    #poly_df = pd.read_csv(poly_file)
    
    idx_array = running_idx+data_df.index.values
    idx_df = pd.DataFrame(idx_array, columns=['IDX'])
    end_idx.append(idx_array[-1])
    running_idx = idx_array[-1] + 1
    
#     zero_indices = list(marker_df[marker_df[M10Y] == 0].index)
#     print(zero_indices)
    
#     data_df = data_df.drop(data_df.index[zero_indices]).reset_index()
#     marker_df = marker_df.drop(marker_df.index[zero_indices])
#     poly_df = poly_df.drop(poly_df.index[zero_indices])
#     idx_df = idx_df.drop(idx_df.index[zero_indices])
    
    data_df_list.append(data_df)
    marker_df_list.append(marker_df)
    #poly_df_list.append(poly_df)
    idx_df_list.append(idx_df)

total_data_df = pd.concat(data_df_list).reset_index()
total_marker_df = pd.concat(marker_df_list).reset_index()
#total_poly_df = pd.concat(poly_df_list).reset_index()
total_idx_df = pd.concat(idx_df_list).reset_index()


#training_data_list = [total_data_df, total_marker_df, total_poly_df, total_idx_df]
training_data_list = [total_data_df, total_marker_df, total_idx_df]
all_df = pd.concat(training_data_list, axis=1)

x_labels = ["M1-PL", "M1-PR", "M2-PL", "M2-PR"]
# NOTE: Uncomment if using polynomial
# y_labels = ["a0", "a1", "a2", "a3", "a4", "a5", "d"]

# NOTE: Uncomment if using markers
y_labels = ["M1X", "M1Y"]
# y_labels = ["M1X", "M1Y", "M2X", "M2Y", "M3X", "M3Y",
#             "M4X", "M4Y", "M5X", "M5Y", "M6X", "M6Y",
#             "M7X", "M7Y", "M8X", "M8Y", "M9X", "M9Y",
#             "M10X", "M10Y"]

idx_labels = ["IDX"]

MX_min = -15
MX_max = 15
MY_min = 0
MY_max = 40

P_min = 95
P_max = 121

def normalize_columns(df, col_list):
    for c in col_list:
        if "X" in c:
            min_val = MX_min
            max_val = MX_max
        elif "Y" in c:
            min_val = MY_min
            max_val = MY_max
        elif "P" in c:
            min_val = P_min
            max_val = P_max
        #print("column: " + str(c) + ", min: " + str(min_val) + ", max: " + str(max_val))
        df[c] = (df[c] - min_val) / (max_val - min_val)
        
normalize_columns(all_df, x_labels)
normalize_columns(all_df, y_labels)

       index   TIME    M1-PL    M1-PR    M2-PL    M2-PR  M1-AL-IN  M1-AL-OUT  \
0        0.0  0.159  101.328  100.797  100.803  100.834       0.0        0.0   
1        1.0  0.661  101.328  100.797  100.809  100.834       0.0        0.0   
2        2.0  1.230  101.328  100.797  100.809  100.828       0.0        0.0   
3        3.0  1.740  101.328  100.797  100.803  100.828       0.0        0.0   
4        4.0  2.233  101.328  100.797  100.809  100.834       0.0        0.0   
...      ...    ...      ...      ...      ...      ...       ...        ...   
11991    NaN    NaN      NaN      NaN      NaN      NaN       NaN        NaN   
11992    NaN    NaN      NaN      NaN      NaN      NaN       NaN        NaN   
11993    NaN    NaN      NaN      NaN      NaN      NaN       NaN        NaN   
11994    NaN    NaN      NaN      NaN      NaN      NaN       NaN        NaN   
11995    NaN    NaN      NaN      NaN      NaN      NaN       NaN        NaN   

       M1-AR-IN  M1-AR-OUT  ...        

In [18]:
# Prepare data for DNN
# shuffle the rows
#train_df = all_df.sample(frac=0.8, random_state=25)
#test_df = all_df.drop(train_df.index)

x_dim = len(x_labels)
y_dim = len(y_labels)
print(x_dim)
print(y_dim)

x_data = all_df[x_labels]
y_data = all_df[y_labels]
idx_data = all_df[idx_labels]

x_data_shuffled, y_data_shuffled, idx_data_shuffled = sklearn.utils.shuffle(x_data, y_data, idx_data)

split = int(0.8 * len(x_data))

x_train = x_data_shuffled[:split]
y_train = y_data_shuffled[:split]
idx_train = idx_data_shuffled[:split]

x_test = x_data_shuffled[split:]
y_test = y_data_shuffled[split:]
idx_test = idx_data_shuffled[split:]
print(y_train)

4
2
            M1X       M1Y
7125   0.501744  0.076171
9426   0.501021  0.068071
2836   0.498171  0.096478
10149       NaN       NaN
2662   0.500610  0.069242
...         ...       ...
6901   0.505383  0.084028
3203   0.500000  0.072474
8614   0.501294  0.067828
10591  0.494419  0.096944
99     0.498781  0.085399

[9596 rows x 2 columns]


In [14]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(9596, 4)
(2400, 4)
(9596, 2)
(2400, 2)


# DNN

In [15]:
fmodel = Sequential()
fmodel.add(Dense(128, input_dim=x_dim, activation='relu'))
fmodel.add(Dropout(0.2))
fmodel.add(Dense(64, activation='relu'))
fmodel.add(Dropout(0.2))
fmodel.add(Dense(32, activation='relu'))
fmodel.add(Dropout(0.2))
fmodel.add(Dense(16, activation='relu'))
fmodel.add(Dropout(0.2))
fmodel.add(Dense(y_dim, activation='relu'))

fmodel.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
fmodel.summary()

bmodel = Sequential()
bmodel.add(Dense(128, input_dim=y_dim, activation='relu'))
bmodel.add(Dropout(0.2))
bmodel.add(Dense(64, activation='relu'))
bmodel.add(Dropout(0.2))
bmodel.add(Dense(32, activation='relu'))
bmodel.add(Dropout(0.2))
bmodel.add(Dense(16, activation='relu'))
bmodel.add(Dropout(0.2))
bmodel.add(Dense(x_dim, activation='relu'))

bmodel.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
bmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               640       
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 16)                5

2022-04-09 17:19:11.066448: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
history = fmodel.fit(x_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
612/612 [==============================] - 1s 458us/step - loss: 0.0829 - accuracy: 0.2756
Epoch 2/100
612/612 [==============================] - 0s 458us/step - loss: 0.0077 - accuracy: 0.6990
Epoch 3/100
612/612 [==============================] - 0s 462us/step - loss: 0.0044 - accuracy: 0.8291
Epoch 4/100
612/612 [==============================] - 0s 460us/step - loss: 0.0035 - accuracy: 0.8471
Epoch 5/100
612/612 [==============================] - 0s 468us/step - loss: 0.0030 - accuracy: 0.8537
Epoch 6/100
612/612 [==============================] - 0s 462us/step - loss: 0.0028 - accuracy: 0.8490
Epoch 7/100
612/612 [==============================] - 0s 467us/step - loss: 0.0025 - accuracy: 0.8713
Epoch 8/100
612/612 [==============================] - 0s 462us/step - loss: 0.0025 - accuracy: 0.8793
Epoch 9/100
612/612 [==============================] - 0s 480us/step - loss: 0.0024 - accuracy: 0.8883
Epoch 10/100
612/612 [==============================] - 0s 459us/step - l

In [16]:
history = bmodel.fit(y_train, x_train, epochs=100, batch_size=8)

Epoch 1/100
1200/1200 [==============================] - 2s 1ms/step - loss: nan - accuracy: 0.4001
Epoch 2/100
1200/1200 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.4003
Epoch 3/100
1200/1200 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.4003
Epoch 4/100
1200/1200 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.4003
Epoch 5/100
1200/1200 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.4003
Epoch 6/100
1200/1200 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.4003
Epoch 7/100
1200/1200 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.4003
Epoch 8/100
1200/1200 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.4003
Epoch 9/100
1200/1200 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.4003
Epoch 10/100
1200/1200 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.4003

KeyboardInterrupt: 

In [ ]:
errors = []
for i,x in x_test.iterrows():
    inp = np.array(x).reshape(1,-1)
    y = model.predict(inp)[0]
    y_true = np.array(y_test.loc[i,:])
    error = 0.0
    for idx in range(len(y)):
        error += ((y[idx] - y_true[idx]) ** 2) * (1/len(y))
    errors.append(error)
mse = sum(errors) / len(errors)
print(mse)
    

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss: %.8f, Accuracy: %.2f' % (loss, (accuracy*100)))

In [ ]:
pickle.dump(history.history["loss"], open("objects/dnn_loss.p", "wb"))

In [ ]:
def rescale_output(y):
    ret = []
    for i in range(len(y)):
        max_val = y_maxes[i]
        min_val = y_mins[i]
        val = y[i]
        rescaled_val = (val * (max_val - min_val)) + min_val
        ret.append(rescaled_val)
    return np.array(ret)

In [ ]:
num_tests = len(x_test)
def evaluate_poly(idx=(0,num_tests-1,1)):
    index = x_test.index[idx]
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    y_pred = model.predict(x_select)
    y_pred = rescale_output(y_pred[0])
    a_pred = np.flip(y_pred[:-1])
    d_pred = y_pred[-1]
    
    y_select = np.array(y_test.loc[index, :]).reshape(1, -1)
    y_select = rescale_output(y_select[0])
    a_select = np.flip(y_select[:-1])
    d_select = y_select[-1]
    
    print(d_pred, d_select)
    
    poly_pred = np.poly1d(a_pred)
    poly_select = np.poly1d(a_select)
    
    yp = np.linspace(0, d_pred)
    xp = poly_pred(yp)
    
    ys = np.linspace(0, d_select)
    xs = poly_select(ys)
    
    plt.plot(xs, ys, label="actual")
    plt.plot(xp, yp, label="predicted")
    plt.xlim([-20,20])
    plt.ylim([-1,39])
    plt.xlabel("x (cm)")
    plt.ylabel("y (cm)")
    plt.legend()
    plt.title("Predicted Shape vs. Actual Shape")
    plt.show()
    

interact(evaluate_poly)

In [10]:
def rescale_input(y):
    ret = []
    for i in range(len(y)):
        max_val = P_max
        min_val = P_min
        val = y[i]
        rescaled_val = (val * (max_val - min_val)) + min_val
        ret.append(rescaled_val)
    return np.array(ret)

def rescale_output(y):
    ret = []
    for i in range(len(y)):
        if (i % 2) == 0:
            max_val = MX_max
            min_val = MX_min
        else:
            max_val = MY_max
            min_val = MY_min
        val = y[i]
        rescaled_val = (val * (max_val - min_val)) + min_val
        ret.append(rescaled_val)
    return np.array(ret)

In [ ]:
num_tests = len(x_test)
def evaluate_markers(idx=(0,num_tests-1,1)):
    index = x_test.index[idx]
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    print(x_select.shape)
    print(rescale_input(x_select))
    y_pred = model.predict(x_select)
    y_pred = rescale_output(y_pred[0])
    
    y_select = np.array(y_test.loc[index, :]).reshape(1, -1)
    y_select = rescale_output(y_select[0])
    
    num_markers = int(len(y_pred) / 2)
    
    x_coord_select = [0]
    y_coord_select = [0]
    x_coord_pred = [0]
    y_coord_pred = [0]
    
    for i in range(num_markers):
        x_idx = i * 2
        y_idx = (i * 2) + 1
        x_coord_select.append(y_select[x_idx])
        y_coord_select.append(y_select[y_idx])
        x_coord_pred.append(y_pred[x_idx])
        y_coord_pred.append(y_pred[y_idx])
    
    plt.plot(x_coord_select, y_coord_select, "-o", label="actual")
    plt.plot(x_coord_pred, y_coord_pred, "-o", label="predicted")
    plt.xlim([-20,20])
    plt.ylim([-1,39])
    plt.xlabel("x (cm)")
    plt.ylabel("y (cm)")
    plt.legend()
    plt.title("Predicted Markers vs. Actual Markers")
    plt.show()
        
        
interact(evaluate_markers)

In [15]:
num_tests = len(x_test)
def evaluate(idx=(0,num_tests-1,1)):
    #clear_output(wait=True)
    index = x_test.index[idx]
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    y_pred = fmodel.predict(x_select)
    y_pred = rescale_output(y_pred[0])
    
    y_select = np.array(y_test.loc[index, :]).reshape(1, -1)
    y_select = rescale_output(y_select[0])
    
    num_markers = int(len(y_pred) / 2)
    
    x_coord_select = [0]
    y_coord_select = [0]
    x_coord_pred = [0]
    y_coord_pred = [0]
    
    for i in range(num_markers):
        x_idx = i * 2
        y_idx = (i * 2) + 1
        x_coord_select.append(y_select[x_idx])
        y_coord_select.append(y_select[y_idx])
        x_coord_pred.append(y_pred[x_idx])
        y_coord_pred.append(y_pred[y_idx])
        
    print(x_coord_select)
        
    index = y_test.index[idx]
    y_select = np.array(y_test.loc[index, :]).reshape(1, -1)
    x_pred = bmodel.predict(y_select)
    x_pred = rescale_input(x_pred[0])
    
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    x_select = rescale_input(x_select[0])
    
    print(y_select)
    m1 = [0,1]
    m2 = [2,3]
    
    plt.figure(1)
    plt.plot(x_coord_select, y_coord_select, "-o", label="actual")
    plt.plot(x_coord_pred, y_coord_pred, "-o", label="predicted")
    plt.xlim([-20,20])
    plt.ylim([-1,39])
    plt.xlabel("x (cm)")
    plt.ylabel("y (cm)")
    plt.legend()
    plt.title("Predicted Markers vs. Actual Markers")
    plt.show()
    
    plt.figure(2)
    plt.plot(m1, x_select[:2], "-x", label="m1 left actual", color="C0")
    plt.plot(m1, x_pred[:2], "-x", label="m1 right predicted", color ="C1")
    plt.plot(m2, x_select[2:], "-o", label="m2 left actual", color="C0")
    plt.plot(m2, x_pred[2:], "-o", label="m2 right predicted", color="C1")
    plt.ylabel("kPa")
    plt.xlim([-1,4])
    plt.ylim([95,125])
    plt.legend()
    plt.title("Predicted Pressures vs. Actual Pressures")
    plt.show()
    plt.savefig("pressures.png")
    
    plt.figure(3)
    plt.plot(x_coord_select, y_coord_select, "-o", label="actual")
    plt.xlim([-20,20])
    plt.ylim([-1,39])
    plt.xlabel("x (cm)")
    plt.ylabel("y (cm)")
    plt.legend()
    plt.title("Input Shape")
    plt.show()
    plt.savefig("markers_in.png")
    
    plt.figure(4)
    plt.plot(m1, x_select[:2], "-x", label="m1 left actual", color="C0")
    plt.plot(m2, x_select[2:], "-o", label="m2 left actual", color="C0")
    plt.ylabel("kPa")
    plt.xlim([-1,4])
    plt.ylim([95,125])
    plt.legend()
    plt.title("Input Pressure Values")
    plt.show()
    plt.savefig("pressures_in.png")
        
interact(evaluate)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.evaluate(idx=(0, 1223, 1))>

<Figure size 1200x800 with 0 Axes>

In [ ]:
fig.savefig("test.png")

In [ ]:
model.save("./models/first_model")

In [ ]:
num_tests = len(x_test)
def evaluate_pressures(idx=(0,num_tests-1,1)):
    index = x_test.index[idx]
    print(x_test)
    x_select = np.array(x_test.loc[index, :]).reshape(1,-1)
    print(x_select)
interact(evaluate_pressures)

In [ ]:
# First model tried
model = Sequential()
model.add(Dense(12, input_dim=x_dim, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(y_dim, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [ ]:
num_tests = len(x_test)
def evaluate_pressures(idx=(0,num_tests-1,1)):
    index = x_test.index[idx]
    print(y_test)
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    y_pred = model.predict(x_select)
    y_pred = rescale_input(y_pred[0])
    
    y_select = np.array(y_test.loc[index, :]).reshape(1, -1)
    y_select = rescale_input(y_select[0])
    
    print(y_select)
    m1 = [0,1]
    m2 = [2,3]
    
    plt.plot(m1, y_select[:2], "-x", label="m1 left actual", color="red")
    plt.plot(m1, y_pred[:2], "-x", label="m1 right predicted", color ="blue")
    plt.plot(m2, y_select[2:], "-x", label="m2 left actual", color="red")
    plt.plot(m2, y_pred[2:], "-x", label="m2 right predicted", color="blue")
    plt.ylabel("kPa")
    plt.xlim([-1,4])
    plt.ylim([95,125])
    plt.legend()
    plt.title("Predicted Pressures vs. Actual Pressures")
    plt.show()
        
        
interact(evaluate_pressures)

In [ ]:
# BAD normalize method
def normalize_columns(df, col_list):
    min_vals = []
    max_vals = []
    for c in col_list:
        min_val = df[c].min()
        max_val = df[c].max()
        min_vals.append(min_val)
        max_vals.append(max_val)
        print("column: " + str(c) + ", min: " + str(min_val) + ", max: " + str(max_val))
        df[c] = (df[c] - min_val) / (max_val - min_val)
    return min_vals, max_vals